# Few-Shot Prompts and Example Selectors

• Find the  [Notebook](https://colab.research.google.com/github/towardsai/ragbook-notebooks/blob/main/notebooks/Chapter%2006%20-%20Getting_the_Best_of_Few_Shot_Prompts_and_Example_Selectors.ipynb)  for this section at  [towardsai.net/book](http://towardsai.net/book).

We’ll cover how few-shot prompts and example selectors can enhance the performance of language models in LangChain. While various methods can be used to implement  **few-shot prompting**  and  **example selectors**  in LangChain, we’ll discuss three distinct approaches, examining their advantages and disadvantages.

In [12]:
import os
from langchain_custom_utils.helper import get_openai_api_key, get_activeloop_api_key, print_response
OPENAI_API_KEY = get_openai_api_key()
ACTIVELOOP_API_KEY = get_activeloop_api_key()

## Alternating Human/AI Messages

Using few-shot prompting with alternating human and AI messages is particularly useful for chat-based applications. This technique requires the language model to understand the conversational context and respond appropriately.

Although this strategy is effective in managing conversational contexts and straightforward to implement, its flexibility is limited to chat-based applications. Despite this, alternating human/AI messages can be creatively employed. In this approach, you are essentially writing the chatbot’s responses in your own words and using them as input for the model.

For example, we can create a chat prompt that translates English into pirate language by showing an example to the model using `AIMessagePromptTemplate`. Below is a code snippet illustrating it:

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain import LLMChain
from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    AIMessagePromptTemplate,
    HumanMessagePromptTemplate,
)

chat = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)

template="You are a helpful assistant that translates english to pirate."
system_message_prompt = SystemMessagePromptTemplate.from_template(template)
example_human = HumanMessagePromptTemplate.from_template("Hi")
example_ai = AIMessagePromptTemplate.from_template("Argh me mateys")
human_template="{text}"
human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)

chat_prompt = ChatPromptTemplate.from_messages([system_message_prompt, example_human, example_ai, human_message_prompt])
chain = LLMChain(llm=chat, prompt=chat_prompt)
chain.run("I love programming.")

## Few-Shot Prompting

Few-shot prompting can improve the output quality as the model better understands the task by reviewing the examples. However, using more tokens might lead to less effective results if the examples provided are not carefully chosen or are misleading.

Implementing the few-shot learning technique involves using the `FewShotPromptTemplate` class, which requires a `PromptTemplate` and a set of few-shot examples. The class combines the prompt template with these examples, aiding the language model in producing more accurate responses. LangChain’s `FewShotPromptTemplate` can be used to organize the approach systematically:

In [14]:
from langchain import PromptTemplate, FewShotPromptTemplate

# create our examples
examples = [
    {
        "query": "What's the weather like?",
        "answer": "It's raining cats and dogs, better bring an umbrella!"
    }, {
        "query": "How old are you?",
        "answer": "Age is just a number, but I'm timeless."
    }
]

# create an example template
example_template = """
User: {query}
AI: {answer}
"""

# create a prompt example from above template
example_prompt = PromptTemplate(
    input_variables=["query", "answer"],
    template=example_template
)

# now break our previous prompt into a prefix and suffix
# the prefix is our instructions
prefix = """The following are excerpts from conversations with an AI
assistant. The assistant is known for its humor and wit, providing
entertaining and amusing responses to users' questions. Here are some
examples:
"""
# and the suffix our user input and output indicator
suffix = """
User: {query}
AI: """

# now create the few-shot prompt template
few_shot_prompt_template = FewShotPromptTemplate(
    examples=examples,
    example_prompt=example_prompt,
    prefix=prefix,
    suffix=suffix,
    input_variables=["query"],
    example_separator="\n\n"
)

After creating a template, we pass the example and user query to get the results:

In [ ]:
chain = LLMChain(llm=chat, prompt=few_shot_prompt_template)
chain.run("What's the secret to happiness?")

This approach provides enhanced control over the **formatting** of examples and is adaptable to various applications. However, it requires manual curation of few-shot examples and may become less efficient when dealing with many examples.

## Example Selectors

An  **example selector**  is a tool that facilitates the selection of examples to add to a few-shot learning prompt. The core objective of few-shot learning is to develop a function that assesses the similarities between classes in the examples and query sets. An example selector can be strategically designed to pick relevant examples accurately reflecting the desired output.

The `ExampleSelector` is crucial in selecting a subset of examples most beneficial for the language model. This selection process helps craft a prompt more likely to produce a high-quality response. The `LengthBasedExampleSelector` is particularly valuable when managing the context window’s length based on the user’s question length. It chooses fewer examples for longer queries and more for shorter ones, ensuring an efficient use of the available context.

This section demonstrates how to use the `LengthBasedExampleSelector` class to efficiently manage the context window and customize prompts with relevant examples.

First, import the required classes:

In [16]:
from langchain.prompts.example_selector import LengthBasedExampleSelector
from langchain.prompts import FewShotPromptTemplate, PromptTemplate

Define your examples and the `example_prompt`:

In [17]:
example = [
    {"word": "happy", "antonym": "sad"},
    {"word": "tall", "antonym": "short"},
    {"word": "energetic", "antonym": "lethargic"},
    {"word": "sunny", "antonym": "gloomy"},
    {"word": "windy", "antonym": "calm"},
]

example_template = """
Word: {word}
Antonym: {antonym}
"""

example_prompt = PromptTemplate(
    input_variables=["word", "antonym"],
    template=example_template
)

Create an instance of `LengthBasedExampleSelector`:

In [18]:
example_selector = LengthBasedExampleSelector(
    examples=example,
    example_prompt=example_prompt,
    max_length=25,
)

Create a `FewShotPromptTemplate` using the `example_selector` variable:

In [19]:
dynamic_prompt = FewShotPromptTemplate(
    example_selector=example_selector,
    example_prompt=example_prompt,
    prefix="Give the antonym of every input",
    suffix="Word: {input}\nAntonym:",
    input_variables=["input"],
    example_separator="\n\n",
)

Generate a sample prompt using the format method to inspect the output:

In [ ]:
print(dynamic_prompt.format(input="black"))

This method effectively handles several examples and provides customization options through different selectors. However, it requires manual curation of examples, which may only be suitable for some applications.

Here is an example of LangChain’s `SemanticSimilarityExampleSelector` to choose examples based on their semantic similarity to the input query. This example demonstrates the steps to create an `ExampleSelector` and formulate a prompt using a few-shot methodology.

In [ ]:
from langchain.prompts.example_selector import SemanticSimilarityExampleSelector
from langchain.vectorstores import DeepLake
from langchain.embeddings import OpenAIEmbeddings
from langchain.prompts import FewShotPromptTemplate, PromptTemplate

# Create a PromptTemplate
example_prompt = PromptTemplate(
    input_variables=["input", "output"],
    template="Input: {input}\nOutput: {output}",
)

# Define some examples
examples = [
    {"input": "0°C", "output": "32°F"},
    {"input": "10°C", "output": "50°F"},
    {"input": "20°C", "output": "68°F"},
    {"input": "30°C", "output": "86°F"},
    {"input": "40°C", "output": "104°F"},
]

# create Deep Lake dataset
my_activeloop_org_id = "jdhernandez" # TODO: use your organization id here
my_activeloop_dataset_name = "langchain_course_fewshot_selector"
dataset_path = f"hub://{my_activeloop_org_id}/{my_activeloop_dataset_name}"
db = DeepLake(dataset_path=dataset_path, overwrite=True)

# Embedding function
embeddings = OpenAIEmbeddings(model="text-embedding-ada-002")

# Instantiate SemanticSimilarityExampleSelector using the examples
example_selector = SemanticSimilarityExampleSelector.from_examples(
    examples, embeddings, db, k=1
)

# Create a FewShotPromptTemplate using the example_selector
similar_prompt = FewShotPromptTemplate(
    example_selector=example_selector,
    example_prompt=example_prompt,
    prefix="Convert the temperature from Celsius to Fahrenheit",
    suffix="Input: {temperature}\nOutput:",
    input_variables=["temperature"],
)

# Test the similar_prompt with different inputs
print(similar_prompt.format(temperature="10°C"))   # Test with an input
print(similar_prompt.format(temperature="30°C"))  # Test with another input

# Add a new example to the SemanticSimilarityExampleSelector
similar_prompt.example_selector.add_example({"input": "50°C", "output": "122°F"})
print(similar_prompt.format(temperature="40°C")) # Test with a new input after adding the example

The `SemanticSimilarityExampleSelector` employs the Deep Lake vector database and `OpenAIEmbeddings` to assess semantic similarity. This tool stores examples in a cloud-based database (e.g., Deep Lake) and retrieves semantically similar samples.

In our process, we first constructed a prompt template and included several examples related to temperature conversions. Following this, we initiated the semantic similarity example selector and created a few-shot prompt template using the selector, `example_prompt`, and the designated prefix and suffix. By utilizing the semantic similarity example selector in combination with the few-shot prompt template, we created dynamic and task-specific context-aware prompts. These tools offer a flexible and adaptable way to generate prompts, enabling the use of language models for a diverse range of tasks.